In [5]:
!wget https://fisd-dataset.s3.amazonaws.com/fisd-asanti-twi-90p.zip

--2024-12-12 00:23:30--  https://fisd-dataset.s3.amazonaws.com/fisd-asanti-twi-90p.zip
Resolving fisd-dataset.s3.amazonaws.com (fisd-dataset.s3.amazonaws.com)... 52.217.73.44, 3.5.28.208, 54.231.192.161, ...
Connecting to fisd-dataset.s3.amazonaws.com (fisd-dataset.s3.amazonaws.com)|52.217.73.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233403440 (223M) [application/zip]
Saving to: ‘fisd-asanti-twi-90p.zip’

fisd-asanti-twi-90p 100%[===================>] 222.59M  4.60MB/s    in 27s     

2024-12-12 00:23:58 (8.38 MB/s) - ‘fisd-asanti-twi-90p.zip’ saved [233403440/233403440]



In [6]:
!unzip fisd-asanti-twi-90p.zip

Streaming output truncated to the last 5000 lines.
  inflating: fisd-asanti-twi-90p/audios/AsantiTwiMa19-ENzuHlXS-Tmp133-NMBnll.ogg  
  inflating: __MACOSX/fisd-asanti-twi-90p/audios/._AsantiTwiMa19-ENzuHlXS-Tmp133-NMBnll.ogg  
  inflating: fisd-asanti-twi-90p/audios/AsantiTwiFm30-PAg4K8fA-Tmp014-Qw0mR9.ogg  
  inflating: __MACOSX/fisd-asanti-twi-90p/audios/._AsantiTwiFm30-PAg4K8fA-Tmp014-Qw0mR9.ogg  
  inflating: fisd-asanti-twi-90p/audios/AsantiTwiMa40-PAQ7Eoc7-Tmp042-jzbBjb.ogg  
  inflating: __MACOSX/fisd-asanti-twi-90p/audios/._AsantiTwiMa40-PAQ7Eoc7-Tmp042-jzbBjb.ogg  
  inflating: fisd-asanti-twi-90p/audios/AsantiTwiMa38-RAtJ0ACM-Tmp128-D8MJNw.ogg  
  inflating: __MACOSX/fisd-asanti-twi-90p/audios/._AsantiTwiMa38-RAtJ0ACM-Tmp128-D8MJNw.ogg  
  inflating: fisd-asanti-twi-90p/audios/AsantiTwiMa25-JAFefgqX-Tmp081-hvwUo8.ogg  
  inflating: __MACOSX/fisd-asanti-twi-90p/audios/._AsantiTwiMa25-JAFefgqX-Tmp081-hvwUo8.ogg  
  inflating: fisd-asanti-twi-90p/audios/AsantiTwiFm24-BDlTV6mS-T

In [8]:
%%capture
!pip install datasets

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import os
from datasets import Dataset

In [11]:
!pip install pandas datasets transformers

import os
import pandas as pd
import re
import unicodedata
from datasets import Dataset, DatasetDict

#storing paths for csv and audio is variables
train_csv_path = "/content/fisd-asanti-twi-90p/data.csv"
audio_base_path = "/content/fisd-asanti-twi-90p/audios/"

##########################################
# Define cleaning function for training
##########################################
def clean_text_train(text, allowed_symbols=None):
    """
    Cleans the input text by:
    - Converting to uppercase.
    - Normalizing Unicode characters.
    - Removing non-alphanumeric characters except those specified in allowed_symbols.
    - Stripping extra whitespace.

    Args:
        text (str): The text to clean.
        allowed_symbols (set): A set of symbols to retain in the text.

    Returns:
        str: The cleaned text.
    """
    # Convert to uppercase
    #text = text.upper()

    # Normalize Unicode to NFC
    text = unicodedata.normalize("NFC", text)

    # Define default allowed characters if none provided
    if allowed_symbols is None:
        allowed_symbols = set("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789ƆƐabcdefghijklmnopqrstuvwxyzɛɔ ")

    # Create a set of allowed characters (letters, digits, space, and allowed symbols)
    allowed_chars = set("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789ƆƐabcdefghijklmnopqrstuvwxyzɛɔ").union(allowed_symbols)

    # Remove characters not in allowed_chars
    cleaned = "".join(ch for ch in text if ch in allowed_chars)

    # Remove extra whitespace
    cleaned = re.sub(r"\s+", " ", cleaned).strip()

    return cleaned


# Load training CSV file

train_df = pd.read_csv(train_csv_path, delimiter="\t", on_bad_lines='skip')

# Remove 'Unnamed: 0' column if it exists
train_df = train_df.drop(['Unnamed: 0'], axis=1, errors='ignore')

# Print a few rows before cleaning for verification
print("=== Sample Rows Before Cleaning ===")
print(train_df.head(5))


# Update 'Audio Filepath' by removing the lacounda
train_df['Audio Filepath'] = train_df['Audio Filepath'].apply(
    lambda x: os.path.join(audio_base_path, os.path.basename(x))
)


# Clean the 'Transcription' column
# Specify allowed symbols (add any symbols you want to retain)
# Example: periods, commas, percent, ampersand, hyphen, Ghanaian Cedi symbol
allowed_symbols_train = set(".,")

train_df['Transcription'] = train_df['Transcription'].apply(
    lambda x: clean_text_train(x, allowed_symbols=allowed_symbols_train) if isinstance(x, str) else x
)

# Printing a few rows after cleaning to verify the cleaning step
print("\n=== Sample Rows After Cleaning ===")
print(train_df.head(5))


# Shuffling and splitting the dataset

shuffled_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
split_ratio = 0.8  # 80% for training
split_index = int(len(shuffled_df) * split_ratio)

train_df = shuffled_df[:split_index]       # Training data
validation_df = shuffled_df[split_index:]  # Validation data


# Converting to match Hugging Face Datasets

train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)

common_voice = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset
})

print(common_voice)


# Printing a few samples from the train dataset for final verification

print("\n=== Sample Training Examples ===")
for i in range(min(3, len(train_dataset))):
    print(f"Example {i+1}")
    print("Audio Filepath:", train_dataset[i]["Audio Filepath"])
    print("Transcription:", train_dataset[i]["Transcription"])
    print()


# Print a few samples from the validation dataset for final verification

if len(validation_dataset) > 0:
    print("\n=== Sample Validation Examples ===")
    for i in range(min(3, len(validation_dataset))):
        print(f"Example {i+1}")
        print("Audio Filepath:", validation_dataset[i]["Audio Filepath"])
        print("Transcription:", validation_dataset[i]["Transcription"])
        print()
else:
    print("\nNo validation examples found.")


=== Sample Rows Before Cleaning ===
                                      Audio Filepath        Transcription  \
0  lacuna-audios-train/asanti-twi/audios/AsantiTw...  Maa wɔn ho yɛɛ huam   
1  lacuna-audios-train/asanti-twi/audios/AsantiTw...  Maa wɔn ho yɛɛ huam   
2  lacuna-audios-train/asanti-twi/audios/AsantiTw...  Maa wɔn ho yɛɛ huam   
3  lacuna-audios-train/asanti-twi/audios/AsantiTw...  Maa wɔn ho yɛɛ huam   
4  lacuna-audios-train/asanti-twi/audios/AsantiTw...  Maa wɔn ho yɛɛ huam   

              Translation  
0  For them to smell good  
1  For them to smell good  
2  For them to smell good  
3  For them to smell good  
4  For them to smell good  

=== Sample Rows After Cleaning ===
                                      Audio Filepath    Transcription  \
0  /content/fisd-asanti-twi-90p/audios/AsantiTwiM...  Maawɔnhoyɛɛhuam   
1  /content/fisd-asanti-twi-90p/audios/AsantiTwiM...  Maawɔnhoyɛɛhuam   
2  /content/fisd-asanti-twi-90p/audios/AsantiTwiM...  Maawɔnhoyɛɛhuam   
3  /c

In [12]:
#looking at the features after clean up
common_voice["train"].features

{'Audio Filepath': Value(dtype='string', id=None),
 'Transcription': Value(dtype='string', id=None),
 'Translation': Value(dtype='string', id=None)}

In [13]:
from datasets import Audio

# If you don't need the Translation column, remove it
common_voice = common_voice.remove_columns(["Translation"])

# Rename columns to match the source format
common_voice = common_voice.rename_column("Audio Filepath", "audio")
common_voice = common_voice.rename_column("Transcription", "sentence")

# Casting the audio column to an Audio feature.
# Specifying the sampling rate of 16kHz
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

# Printing the features to confirm
print(common_voice["train"].features)


{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'sentence': Value(dtype='string', id=None)}


In [14]:
import os
from datasets import Audio
#clearing bad files that will cause errors:

# Names of the bad files
bad_files = [
    "AsantiTwiFm20-A SLRKMb-Tmp012-ql0LnX.ogg",
    "AsantiTwiFm20-A SLRKMb-Tmp089-eRlzWZ.ogg",
    "AsantiTwiFm20-A SLRKMb-Tmp011-bcTYyx.ogg",
    "AsantiTwiFm18-MHdyb4ij-Tmp001-6afLtE.ogg",
    "AsantiTwiMa20-SQF23O2h-Tmp014-T6agOK.ogg",

]

# Disabling audio decoding so that filtering won't try to load the audio
common_voice = common_voice.cast_column("audio", Audio(decode=False))

# Filtering out the rows with the bad files
common_voice = common_voice.filter(
    lambda x: os.path.basename(x["audio"]["path"]) not in bad_files
)

# Re-enabling audio decoding after  removing bad files
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000, decode=True))

# Printing to verify the dataset no longer contains those files
print(common_voice)


Filter:   0%|          | 0/22451 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5613 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 22447
    })
    validation: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 5612
    })
})


In [15]:
from transformers import WhisperProcessor

# Do not specify 'language' or set it to None
processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe")

def prepare_dataset(example):
    audio = example["audio"]
    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["sentence"],
    )
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    return example

common_voice = common_voice.map(
    prepare_dataset,
    remove_columns=common_voice["train"].column_names,
    num_proc=1
)

print(common_voice)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Map:   0%|          | 0/22447 [00:00<?, ? examples/s]

Map:   0%|          | 0/5612 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 22447
    })
    validation: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 5612
    })
})


In [16]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any  # This should be an instance of WhisperProcessor

    def __call__(
        self,
        features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        """
        This method takes a batch (list) of examples as input, each example being a dictionary
        containing at least 'input_features' and 'labels'. It returns a dictionary of padded
        tensors ready for model input.
        """

        # Extract the input features (audio features). Each example has 'input_features' as a list with one element.
        input_features = [{"input_features": f["input_features"][0]} for f in features]

        # Pad the input features to a common length using the processor's feature extractor.
        # This creates a batch with 'input_features' as a padded tensor (mel spectrogram for Whisper).
        batch = self.processor.feature_extractor.pad(
            input_features,
            return_tensors="pt"
        )

        # Extract labels. Each example has a 'labels' field, which is a list of token IDs.
        label_features = [{"input_ids": f["labels"]} for f in features]

        # Pad the labels using the processor’s tokenizer. This returns a dictionary with
        # 'input_ids' and 'attention_mask' for the labels.
        labels_batch = self.processor.tokenizer.pad(
            label_features,
            return_tensors="pt"
        )

        # Replace padding token IDs with -100 so that they are ignored by the loss function.
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # If all sequences start with a BOS (beginning-of-sequence) token, remove it.
        # This is sometimes done to align with how the model expects input.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        # Add the processed labels to the batch.
        batch["labels"] = labels

        return batch


In [17]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [18]:
!pip install jiwer
!pip install evaluate

import evaluate

# Load the Word Error Rate (WER) metric
metric = evaluate.load("wer")

# If you have a custom normalizer for Twi, define it here.
import numpy as np
import os
import re
import string
import unicodedata
import torch
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import WhisperProcessor
from datasets import Audio, Dataset, DatasetDict


# Initialising Processor and Metric
# Assuming we have chosen a multilingual model capable of Twi and English code switching to train it on our twi dataset.
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    task="transcribe"  # no language specified, letting Whisper detect language
)

metric = evaluate.load("wer")

#Defining the Twi Financial Normalizer

def twi_financial_normalizer(text: str) -> str:
    text = text.lower()
    text = unicodedata.normalize("NFC", text)

    # Allowed characters include letters, digits, financial symbols, punctuation used in financial contexts
    allowed_chars = set("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789ƆƐ")
    text = "".join(ch for ch in text if ch in allowed_chars)
    text = re.sub(r"\s+", " ", text).strip()
    return text


# Step 4: Define the Compute Metrics Function

# def compute_metrics(pred):
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # Replace -100 with pad_token_id
#     label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

#     # Decode predictions and references
#     pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

#     # Compute orthographic WER (no normalization)
#     wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

#     # Normalize predictions and references
#     pred_str_norm = [twi_financial_normalizer(p) for p in pred_str]
#     label_str_norm = [twi_financial_normalizer(l) for l in label_str]

#     # Filter out empty references to avoid errors in WER computation
#     pred_str_norm = [
#         pred_str_norm[i] for i in range(len(pred_str_norm))
#         if len(label_str_norm[i]) > 0
#     ]
#     label_str_norm = [
#         label_str_norm[i] for i in range(len(label_str_norm))
#         if len(label_str_norm[i]) > 0
#     ]

#     # Compute WER on normalized text
#     wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

#     return {"wer_ortho": wer_ortho, "wer": wer}


def compute_metrics(pred):
    # Extracting predictions and labels
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replacing -100 with the pad token id so that we decode properly
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Decoding predictions and references
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Normalizing predictions and references
    pred_str_norm = [twi_financial_normalizer(p) for p in pred_str]
    label_str_norm = [twi_financial_normalizer(l) for l in label_str]

    # Filtering out empty references to avoid errors in WER computation
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm))
        if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i] for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]



    # Computing orthographic WER (no normalization)
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    cer = 100 * compute_character_error_rate(pred_str, label_str)

    # Computing WER on normalized text
    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer, "cer": cer}

    # Computing CER (Character Error Rate)
    # We define a helper function to compute CER for a list of predictions and references
def compute_character_error_rate(predictions, references):
    """
    Compute the Character Error Rate (CER) between a list of predictions and references.
    CER is defined as the edit distance between the predicted and reference text,
    divided by the number of characters in the reference, multiplied by 100.
    """
    total_distance = 0
    total_chars = 0
    for pred, ref in zip(predictions, references):
        # Remove extra spaces if needed and measure distance
        pred = pred.strip()
        ref = ref.strip()

        # Compute edit distance for characters
        dist = levenshtein_distance(ref, pred)
        total_distance += dist
        total_chars += len(ref)

    # Avoid division by zero if some references are empty
    if total_chars == 0:
        return 0.0

    return total_distance / total_chars


def levenshtein_distance(s1, s2):
    """
    Compute the Levenshtein distance (edit distance) between two strings at the character level.
    This is a standard dynamic programming approach:
    - distance[i][j] represents the edit distance between s1[:i] and s2[:j].
    """
    m = len(s1)
    n = len(s2)

    # Initialize a (m+1)x(n+1) matrix
    distance = np.zeros((m+1, n+1), dtype=int)

    # Base cases: distance from empty string
    for i in range(m+1):
        distance[i][0] = i
    for j in range(n+1):
        distance[0][j] = j

    # Compute distances
    for i in range(1, m+1):
        for j in range(1, n+1):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            distance[i][j] = min(
                distance[i-1][j] + 1,     # deletion
                distance[i][j-1] + 1,     # insertion
                distance[i-1][j-1] + cost # substitution
            )

    return distance[m][n]
    return {"wer": wer, "cer": cer}



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00


In [19]:
#This is the final code for compute_metrics:
def compute_metrics(pred):
    # Extracting predictions and labels
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replacing -100 with the pad token id so that we decode properly
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode predictions and references
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Normalising predictions and references
    pred_str_norm = [twi_financial_normalizer(p) for p in pred_str]
    label_str_norm = [twi_financial_normalizer(l) for l in label_str]

    # Filtering out empty references to avoid errors in WER computation
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm))
        if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i] for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]
    # Ensuring to initialize metrics dictionary
    metrics = {}

    # Computing orthographic WER (no normalization)
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)
    metrics["wer_ortho"] = wer_ortho # Add metric to dictionary

    # Computing CER
    cer = 100 * compute_character_error_rate(pred_str, label_str)
    metrics["cer"] = cer # Add metric to dictionary

    # Computing WER on normalized text
    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)
    metrics["wer"] = wer # Add metric to dictionary

    return metrics # Return the metrics dictionary

In [20]:
!pip install jiwer
!pip install evaluate

import evaluate

# Loading the Word Error Rate (WER) metric
metric = evaluate.load("wer")

# Defing our custon twi normaliser around here:
import numpy as np
import os
import re
import string
import unicodedata
import torch
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import WhisperProcessor
from datasets import Audio, Dataset, DatasetDict


# Initialising Processor and Metric

# Assume you have chosen a multilingual model capable of Twi and English code switching.
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    task="transcribe"  # no language specified, letting Whisper detect language
)

metric = evaluate.load("wer")


# Defining the Twi Financial Normalizer

def twi_financial_normalizer(text: str) -> str:
    text = text.lower()
    text = unicodedata.normalize("NFC", text)

    # Allowed characters include letters, digits, financial symbols, punctuation used in financial contexts
    allowed_chars = set("ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789ƆƐ")
    text = "".join(ch for ch in text if ch in allowed_chars)
    text = re.sub(r"\s+", " ", text).strip()
    return text


# Step 4: Define the Compute Metrics Function

# def compute_metrics(pred):
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # Replace -100 with pad_token_id
#     label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

#     # Decode predictions and references
#     pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

#     # Compute orthographic WER (no normalization)
#     wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

#     # Normalize predictions and references
#     pred_str_norm = [twi_financial_normalizer(p) for p in pred_str]
#     label_str_norm = [twi_financial_normalizer(l) for l in label_str]

#     # Filter out empty references to avoid errors in WER computation
#     pred_str_norm = [
#         pred_str_norm[i] for i in range(len(pred_str_norm))
#         if len(label_str_norm[i]) > 0
#     ]
#     label_str_norm = [
#         label_str_norm[i] for i in range(len(label_str_norm))
#         if len(label_str_norm[i]) > 0
#     ]

#     # Compute WER on normalized text
#     wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

#     return {"wer_ortho": wer_ortho, "wer": wer}


# def compute_metrics(pred):
#     # Extract predictions and labels
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # Replace -100 with the pad token id so that we decode properly
#     label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

#     # Decode predictions and references
#     pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

#     # Normalize predictions and references
#     pred_str_norm = [twi_financial_normalizer(p) for p in pred_str]
#     label_str_norm = [twi_financial_normalizer(l) for l in label_str]

#     # Filter out empty references to avoid errors in WER computation
#     pred_str_norm = [
#         pred_str_norm[i] for i in range(len(pred_str_norm))
#         if len(label_str_norm[i]) > 0
#     ]
#     label_str_norm = [
#         label_str_norm[i] for i in range(len(label_str_norm))
#         if len(label_str_norm[i]) > 0
#     ]

# def compute_character_error_rate(predictions, references):
#     """
#     Compute the Character Error Rate (CER) between a list of predictions and references.
#     CER is defined as the edit distance between the predicted and reference text,
#     divided by the number of characters in the reference, multiplied by 100.
#     """
#     total_distance = 0
#     total_chars = 0
#     for pred, ref in zip(predictions, references):
#         # Remove extra spaces if needed and measure distance
#         pred = pred.strip()
#         ref = ref.strip()

        # Compute edit distance for characters
#         dist = levenshtein_distance(ref, pred)
#         total_distance += dist
#         total_chars += len(ref)

#     # Avoid division by zero if some references are empty
#     if total_chars == 0:
#         return 0.0

#     return total_distance / total_chars


# def levenshtein_distance(s1, s2):
#     """
#     Compute the Levenshtein distance (edit distance) between two strings at the character level.
#     This is a standard dynamic programming approach:
#     - distance[i][j] represents the edit distance between s1[:i] and s2[:j].
#     """
#     m = len(s1)
#     n = len(s2)

#     # Initialize a (m+1)x(n+1) matrix
#     distance = np.zeros((m+1, n+1), dtype=int)

#     # Base cases: distance from empty string
#     for i in range(m+1):
#         distance[i][0] = i
#     for j in range(n+1):
#         distance[0][j] = j

#     # Compute distances
#     for i in range(1, m+1):
#         for j in range(1, n+1):
#             cost = 0 if s1[i-1] == s2[j-1] else 1
#             distance[i][j] = min(
#                 distance[i-1][j] + 1,     # deletion
#                 distance[i][j-1] + 1,     # insertion
#                 distance[i-1][j-1] + cost # substitution
#             )

#     return distance[m][n]

#     # Compute orthographic WER (no normalization)
#     wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

#     cer = 100 * compute_character_error_rate(pred_str, label_str)

#     # Compute WER on normalized text
#     wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

#     return {"wer_ortho": wer_ortho, "wer": wer, "cer": cer}
#     # Compute CER (Character Error Rate)
#     # We define a helper function to compute CER for a list of predictions and references





In [21]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [22]:
from functools import partial

# disabling cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# setting task for generation and re-enable cache
# removing 'language="sinhalese"' since we're allowing Whisper to detect Twi automatically
model.generate = partial(
    model.generate,
    task="transcribe",
    use_cache=True
)


In [28]:
from functools import partial
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/please",    # directory to save models
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,      # adjust as needed for your GPU resources
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=500, #change back to 400
    #max_steps=500,                      # increase if you have more training resources/time
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500, #after every 500 steps the model at that time
    eval_steps=500, #evaluate the model after 500 steps
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    save_total_limit =15,
    num_train_epochs=2
    # push_to_hub=True,                   # Set to False if you do not want to push to Hugging Face Hub
)

# disabling cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# setting task for generation and re-enable cache
# We do not specify language here to allow automatic language detection
model.generate = partial(
    model.generate,
    task="transcribe",
    use_cache=True
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
from transformers import Seq2SeqTrainer
#using the Seq2Seq Trainer since audio is an input sequence and we are outputting a sequence of words
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor
)
#trainer.processing_class = processor.__class__

<ipython-input-32-66356677f495>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [34]:
#training
trainer.train()

Step,Training Loss,Validation Loss,Wer Ortho,Cer,Wer
500,0.011900,0.022285,16.928011,7.098547,0.819672
1000,0.013000,0.012108,8.588738,5.583127,1.229508
1500,0.011400,0.009276,10.851746,4.834279,3.688525
2000,0.001200,0.011020,6.397006,9.758655,0.819672
2500,0.008200,0.009461,5.630791,3.447359,0.819672


Streaming output truncated to the last 5000 lines.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Tra

Step,Training Loss,Validation Loss,Wer Ortho,Cer,Wer
500,0.011900,0.022285,16.928011,7.098547,0.819672
1000,0.013000,0.012108,8.588738,5.583127,1.229508
1500,0.011400,0.009276,10.851746,4.834279,3.688525
2000,0.001200,0.011020,6.397006,9.758655,0.819672
2500,0.008200,0.009461,5.630791,3.447359,0.819672


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=2806, training_loss=0.012404555626112652, metrics={'train_runtime': 21865.5689, 'train_samples_per_second': 2.053, 'train_steps_per_second': 0.128, 'total_flos': 1.296498769477632e+19, 'train_loss': 0.012404555626112652, 'epoch': 2.0})

In [ ]:
# If resuming from checkpoint if it terminates unexpectedly
trainer.train(resume_from_checkpoint=True)